In [10]:
import os
import json
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [11]:
path_100 ="Truck-and-Drone (Unlimited Drone Fleet) Matheuristic Results - Larger Instances.csv"
path_75 ="Truck-and-Drone (Unlimited Drone Fleet) Matheuristic Results - Larger Instances.csv"
path_50 = "Truck-and-Drone (Limited Drone Fleet) Matheuristic Results - Larger Instances.csv"
path_40 = "Truck-and-Drone (Limited Drone Fleet) Matheuristic Results - Larger Instances.csv"
path_30 = "Truck-and-Drone (Limited Drone Fleet) Matheuristic Results - Larger Instances.csv"
path_20 = "Truck-and-Drone (Limited Drone Fleet) Matheuristic Results - 15 and 20 customers.csv"
path_15 = "Truck-and-Drone (Limited Drone Fleet) Matheuristic Results - 15 and 20 customers.csv"
path_10 = "Truck-and-Drone (Limited Drone Fleet) Matheuristic Results - 10 customers.csv"
# Tạo dictionary ánh xạ các giá trị của `a` đến đường dẫn tương ứng
path_dict = {
    100: "Truck-and-Drone (Unlimited Drone Fleet) Matheuristic Results - Larger Instances.csv",
    75: "Truck-and-Drone (Unlimited Drone Fleet) Matheuristic Results - Larger Instances.csv",
    5050: "Truck-and-Drone (Unlimited Drone Fleet) Matheuristic Results - Larger Instances.csv",
    50: "Truck-and-Drone (Limited Drone Fleet) Matheuristic Results - Larger Instances.csv",
    40: "Truck-and-Drone (Limited Drone Fleet) Matheuristic Results - Larger Instances.csv",
    30: "Truck-and-Drone (Limited Drone Fleet) Matheuristic Results - Larger Instances.csv",
    20: "Truck-and-Drone (Limited Drone Fleet) Matheuristic Results - 15 and 20 customers.csv",
    15: "Truck-and-Drone (Limited Drone Fleet) Matheuristic Results - 15 and 20 customers.csv",
    10: "Truck-and-Drone (Limited Drone Fleet) Matheuristic Results - 10 customers.csv"
}

In [12]:
def findOptimal(name, path):
    # Đọc file CSV
    optimal_file = pd.read_csv(path)
    
    # Tách các thành phần từ tên file
    string_component = name.split('_')
    num_cities = int(string_component[2])
    beta = float(string_component[3])
    num = int(string_component[5].split('.')[0])
    center = string_component[6].strip()
    
    
    # Kiểm tra và chuyển đổi kiểu dữ liệu của các cột
    if 'Customers' in optimal_file.columns:
        optimal_file['Customers'] = optimal_file['Customers'].astype(int)
    if 'Beta' in optimal_file.columns:
        optimal_file['Beta'] = optimal_file['Beta'].astype(float)
    if 'Instance' in optimal_file.columns:
        optimal_file['Instance'] = optimal_file['Instance'].astype(int)
    if 'Depot location' in optimal_file.columns:
        optimal_file['Depot location'] = optimal_file['Depot location'].astype(str)
        optimal_file['Depot location'] = optimal_file['Depot location'].str.strip().str.lower()

    
    # Lọc dữ liệu
    filtered_data = optimal_file[
        (optimal_file['Customers'] == num_cities) &
        (optimal_file['Beta'] == beta) &
        (optimal_file['Instance'] == num) &
        (optimal_file['Depot location'].str.lower() == center.lower())
    ]
    
    # Trả về cột 'Objective value' nếu có dữ liệu, nếu không trả về thông báo
    if not filtered_data.empty:
        return filtered_data['Objective value'].iloc[0]
    else:
        return -1


In [13]:
name ="Results_U_10_0.5_Num_1.txt_border_CL1.json"
findOptimal(name, path_10)

149

In [14]:
def get_attribute_in_solution(solution, num_city):
    if num_city <= 20:
        drone_capacity = 2
    else:
        drone_capacity = 10
    if num_city <= 50:
        num_truck = 2
        num_drone = 2
    else:
        num_truck = 2
        num_drone = 20
    limit_drone_time = 90
    num_place_truck_meet_drone = 0
    num_order_drone_cover = 0
    for i in range(len(solution[0])):
        for j in range(len(solution[0][i])):
            if solution[0][i][j][0] != 0:
                if solution[0][i][j][1] != []:
                    num_place_truck_meet_drone += 1
                    num_order_drone_cover += len(solution[0][i][j][1])
    num_drone_flight = len(solution[1])
    return drone_capacity, num_truck, num_drone, limit_drone_time, num_place_truck_meet_drone, num_order_drone_cover, num_drone_flight
    
solution_test = [[[[0, [8, 9, 5, 3]], [9, []], [3, []], [5, []], [7, [7, 4]], [8, []], [4, []]], [[0, []], [10, [10]], [6, [6]], [1, [1, 2]], [2, []]]], 
                 [[[10, [10]]], [[6, [6]]], [[1, [1, 2]]], [[7, [7, 4]]]]]

In [15]:
def find_txt_files(directory):
    """Tìm tất cả các file .txt trong thư mục chỉ định."""
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(".json")]

def read_json_from_txt(file_path):
    """Đọc tất cả các dòng JSON trong file txt và chuyển đổi thành danh sách dict."""
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                json_obj = json.loads(line.strip())
                data.append(json_obj)
            except json.JSONDecodeError:
                print(f"Lỗi khi đọc JSON từ file: {file_path}")
    return data

def process_files(directory, root):
    """Xử lý các file trong thư mục và lưu kết quả tốt nhất vào Excel."""
    # print("hehe")
    txt_files = find_txt_files(directory)
    results = []
    if directory == f"{root}/50_heu":
        num_sheet = "50_heu"
    else:
        num_sheet = int(directory.split('/')[-1])
    
    for file in txt_files:
        filename = os.path.basename(file)
        data = read_json_from_txt(file)
        num_city = filename.split('_')[2]
        if directory == f"{root}/50_heu":
            num_city = 5050
        beta = filename.split('_')[3]
        depot_type = filename.split('_')[6]
        optimal_solution = findOptimal(filename, path_dict[int(num_city)])
        if optimal_solution == -1:
            continue
        filename_short = filename.split('.txt')[0].replace("Results_","")
        filename_short = "_".join([filename_short.split("_")[i] for i in [0, 1, 3, 4]])
        if data:
            # Tìm kết quả tốt nhất theo best_fitness thấp nhất (hoặc cao nhất tùy vào bài toán)
            best_entry = min(data, key=lambda x: x["best_fitness"], default=None)
            average_fitness = sum(x["best_fitness"] for x in data) / len(data) if data else 0
            best_gap = (best_entry["best_fitness"]-optimal_solution)/optimal_solution
            average_gap = (average_fitness-optimal_solution)/optimal_solution
            best_fitness_values = [x["best_fitness"] for x in data]
            solutionnnn = [x["all_solution"] for x in data]
            fitnesssss = [x["all_fitness"] for x in data]
            num_run = [x["Numrun"] for x in data]
            std_fitness = np.std(best_fitness_values, ddof=0)
            drone_capacity, num_truck, num_drone, limit_drone_time, num_place_truck_meet_drone, num_order_drone_cover, num_drone_flight = get_attribute_in_solution(best_entry["best_solution"], int(num_city))
            if best_entry:
                results.append({
                    "File Name": filename_short,
                    "drone_capacity": drone_capacity,
                    "beta": beta,
                    "depot_type": depot_type,
                    "num_truck": num_truck,
                    "num_drone": num_drone,
                    "Best Fitness": best_entry["best_fitness"],
                    "Their best fitness": optimal_solution,
                    "Best GAP": best_gap,
                    "avg": average_fitness,
                    "Std": std_fitness,
                    "Gap Fitness": average_fitness,
                    "Average GAP": average_gap,
                    "Best Solution": json.dumps(best_entry["best_solution"]),
                    "limit_drone_time": limit_drone_time,
                    "num_place_truck_meet_drone": num_place_truck_meet_drone,
                    "num_order_drone_cover": num_order_drone_cover,
                    "num_drone_flight": num_drone_flight,
                    "all_solution": solutionnnn,
                    "all_fitness": fitnesssss,
                    "num_run": num_run
                })
    

    # Ghi vào file Excel
    if results:
        df = pd.DataFrame(results)

        # Chuyển đổi Best Fitness và Best GAP thành %


        # Tính trung bình của Best GAP và Average GAP trên tất cả các file
        average_avg= df["avg"].mean()
        average_fitness = df["Best Fitness"].mean()
        average_best_gap = df["Best GAP"].mean()
        average_average_gap = df["Average GAP"].mean()
        average_std= df["Std"].mean()
        
        
        
        df["Average GAP"] = df["Average GAP"].apply(lambda x: f"{x * 100:.2f}%")
        df["Best GAP"] = df["Best GAP"].apply(lambda x: f"{x * 100:.2f}%")

        # Thêm giá trị trung bình vào dòng cuối cùng trong cột tương ứng
        df.loc[len(df)] = [""] * df.shape[1]  # Thêm dòng trống
        # df.loc[len(df)] = [""] * df.shape[1]  # Thêm dòng trống nếu muốn cách xa một dòng
        # Chuyển đổi giá trị trung bình thành phần trăm
        average_best_gap_percentage = f"{average_best_gap * 100:.2f}%"
        average_average_gap_percentage = f"{average_average_gap * 100:.2f}%"
        
        print(average_std)
        print(average_avg)
        print(average_fitness)
        print(average_best_gap_percentage)
        print(average_average_gap_percentage)


        # Thêm giá trị trung bình vào dòng cuối cùng trong cột tương ứng, dưới dạng phần trăm
        # df.loc[len(df)] = [""] * (df.columns.get_loc("Best GAP")) + [average_best_gap_percentage, ""] + [average_average_gap_percentage] + [""] * (df.shape[1] - df.columns.get_loc("Average GAP") - 1)


        excel_path = f"{root}/best_fitness_results.xlsx"
        
        # Kiểm tra nếu file đã tồn tại
        if os.path.exists(excel_path):
            with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df.to_excel(writer, sheet_name=str(num_sheet), index=False)
        else:
            with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
                df.to_excel(writer, sheet_name=str(num_sheet), index=False)
        
        print(f"Đã lưu kết quả vào sheet {num_sheet} trong file: {excel_path}")
    else:
        print("Không tìm thấy dữ liệu hợp lệ để ghi vào file Excel.")


In [16]:
root = "3600_new_annotation"
folder_paths = [f"{root}/10", f"{root}/15", f"{root}/20", f"{root}/30", f"{root}/40", f"{root}/50", f"{root}/50_heu", f"{root}/100"]
for folder_path in folder_paths:
    if os.path.isdir(folder_path):
        process_files(folder_path, root)
    else:
        print("Đường dẫn không hợp lệ!")

0.35657535461612627
226.01306071006508
225.69516695796932
0.57%
0.74%
Đã lưu kết quả vào sheet 10 trong file: 3600_new_annotation/best_fitness_results.xlsx
0.8411892992706664
255.99186710214425
255.05266775486828
0.59%
1.04%
Đã lưu kết quả vào sheet 15 trong file: 3600_new_annotation/best_fitness_results.xlsx
1.689629213650515
289.85419900753914
287.73761386415794
-1.04%
-0.20%
Đã lưu kết quả vào sheet 20 trong file: 3600_new_annotation/best_fitness_results.xlsx
30.533515758580346
396.9232038359536
374.2179453775227
-1.18%
4.82%
Đã lưu kết quả vào sheet 30 trong file: 3600_new_annotation/best_fitness_results.xlsx
41.38195318469358
492.5113428334328
433.0208157442749
-0.47%
13.01%
Đã lưu kết quả vào sheet 40 trong file: 3600_new_annotation/best_fitness_results.xlsx
47.032565946454845
562.1761085411836
487.24977108228126
-0.18%
15.34%
Đã lưu kết quả vào sheet 50 trong file: 3600_new_annotation/best_fitness_results.xlsx
50.22589329044664
536.3778185834747
458.68061336772723
-2.29%
14.13%


In [ ]:
folder_path = "3600/50"
if os.path.isdir(folder_path):
        process_files(folder_path)

34.131207721407016
544.2026642237827
498.53251446923775
6.16%
15.80%
Đã lưu kết quả vào sheet 50 trong file: best_fitness_results__11.xlsx
